In [15]:
# inspect D:\Life\Academic\Skripsi\Code\data\wikipediasmallLM\data\train-00000-of-00145-437d77a320cee9e2.parquet

import pandas as pd

df = pd.read_parquet("D:\\Life\\Academic\\Skripsi\\Code\\data\\wikipediasmallLM\\data\\train-00000-of-00145-437d77a320cee9e2.parquet")
print(df.head())

   id           title                                               text  \
0   0  Deaths in 2022  The following notable deaths occurred in 2022....   
1   1         YouTube  YouTube is a global online video sharing and s...   
2   2         YouTube  In October 2006, YouTube was bought by Google ...   
3   3         YouTube  Since its purchase by Google, YouTube has expa...   
4   4         YouTube  YouTube has had an unprecedented social impact...   

                                            url   wiki_id        views  \
0  https://en.wikipedia.org/wiki?curid=69407798  69407798  5674.449219   
1   https://en.wikipedia.org/wiki?curid=3524766   3524766  5409.561035   
2   https://en.wikipedia.org/wiki?curid=3524766   3524766  5409.561035   
3   https://en.wikipedia.org/wiki?curid=3524766   3524766  5409.561035   
4   https://en.wikipedia.org/wiki?curid=3524766   3524766  5409.561035   

   paragraph_id  langs                                                emb  
0             0     38

In [13]:
# convert all emb to list
df["emb"] = df["emb"].apply(lambda x: x.tolist())

In [2]:
df['id'].is_unique


True

In [5]:
len(df['id'].unique())

242538

In [9]:
from qdrant_client import QdrantClient
client = QdrantClient(url="http://localhost:6333")
COLLECTION_NAME = "wikipedia"

client.count(COLLECTION_NAME, exact=True)

CountResult(count=1455228)

In [10]:
info = client.get_collection(COLLECTION_NAME)
print(info)

status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=1455228 points_count=1455228 segments_count=5 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=True, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=8, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=True, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=10000, indexing_threshold=10000, flush_interval_sec=5, max_optimization_threads=4), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_confi

In [5]:
client.delete_collection(COLLECTION_NAME)

True

In [14]:
COLLECTION_NAME = "wikipedia"
BATCH_SIZE = 128
PROGRESS_FILE = "progress.json"
VECTOR_SIZE = 384
DATA_DIR = r"D:\Life\Academic\Skripsi\Code\data\wikipediasmallLM\data"
NUM_FILES_TO_PROCESS = 1  # Number of files to process (set to None for all files)

In [ ]:
from qdrant_client import QdrantClient, models
from qdrant_client.models import (
    VectorParams,
    Distance,
    HnswConfigDiff,
    ScalarQuantization,
    ScalarQuantizationConfig,
    ScalarType
)

client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(
        size=VECTOR_SIZE,
        distance=Distance.COSINE,
        on_disk=True
    ),
    hnsw_config=HnswConfigDiff(
        m=8,
        ef_construct=50,
        on_disk=True
    ),
    optimizers_config=models.OptimizersConfigDiff(
        memmap_threshold=10000,
        indexing_threshold=20000
    ),
    quantization_config=ScalarQuantization(
        scalar=ScalarQuantizationConfig(
            type=ScalarType.INT8,
            quantile=0.999,
            always_ram=False
        )
    )
)

True

In [16]:
import json
try:
    with open(PROGRESS_FILE, "r") as f:
        progress_data = json.load(f)
    # Initialize last_file_index if not present
    if "last_file_index" not in progress_data:
        progress_data["last_file_index"] = -1
except FileNotFoundError:
    progress_data = {"last_file_index": -1, "file_progress": {}}

In [20]:
import glob, os
files = sorted(glob.glob(os.path.join(DATA_DIR, "*.parquet")))
print(f"Found {len(files)} parquet files.")

Found 6 parquet files.


In [21]:
start_index = progress_data["last_file_index"] + 1
start_index

0

In [24]:
if NUM_FILES_TO_PROCESS is None:
    end_index = len(files)  # Process all remaining files
else:
    end_index = min(start_index + NUM_FILES_TO_PROCESS, len(files))

if start_index >= end_index:
    print("All files already processed. Nothing to do.")
    exit()

In [27]:
files_to_process = files[start_index:end_index]
files_to_process

['D:\\Life\\Academic\\Skripsi\\Code\\data\\wikipediasmallLM\\data\\train-00000-of-00145-437d77a320cee9e2.parquet']

In [29]:
print(f"Processing files {start_index} to {end_index-1} ({len(files_to_process)} files)")

Processing files 0 to 0 (1 files)
